In [1]:
import requests
import os
import ast
import re
from transformers import AutoTokenizer
from transformers.utils import get_json_schema
from langchain_community.llms import LlamaCpp
from dotenv import load_dotenv
load_dotenv(os.getcwd()+"/.env", override=True)

False

In [10]:
TOKENIZER = AutoTokenizer.from_pretrained("ibm-granite/granite-3.0-8b-instruct")

tokenizer_config.json:   0%|          | 0.00/5.64k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

In [2]:
FNAME = "granite-3.0-8b-instruct-Q4_K_M.gguf"

if not os.path.exists(FNAME):
    res = requests.get('https://huggingface.co/bartowski/granite-3.0-8b-instruct-GGUF/resolve/main/granite-3.0-8b-instruct-Q4_K_M.gguf')
    with open(FNAME, 'wb') as file:
        file.write(res.content)

In [3]:
AV_STOCK_API_KEY = os.getenv('AV_STOCK_API_KEY',"TRIITLN2AW82X8LH")
WEATHER_API_KEY = os.getenv('WEATHER_API_KEY',"e43e2793e6eccc0fabf9ca3d4cf0cb0f")

In [4]:
def get_stock_price(ticker: str, date: str) -> dict:
    """
    Retrieves the lowest and highest stock prices for a given ticker and date.
    Args:
    ticker: The stock ticker symbol, e.g., "IBM".
    date: The date in "YYYY-MM-DD" format for which you want to get stock prices.
    Returns:
    A dictionary containing the low and high stock prices on the given date.
    """
    print(f"Getting stock price for {ticker} on {date}")
    try:
        stock_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={AV_STOCK_API_KEY}"
        stock_data = requests.get(stock_url)
        stock_low = stock_data.json()["Time Series (Daily)"][date]["3. low"]
        stock_high = stock_data.json()["Time Series (Daily)"][date]["2. high"]
        return {
            "low": stock_low,
            "high": stock_high
        }
    except Exception as e:
        print(f"Error fetching stock data: {e}")
        return {
            "low": "none",
            "high": "none"
        }

In [5]:
def get_current_weather(location: str) -> dict:
    """
    Fetches the current weather for a given location (default: San Francisco).

    Args:
        location: The name of the city for which to retrieve the weather information.

    Returns:
        A dictionary containing weather information such as temperature, weather description, and humidity.
    """
    print(f"Getting current weather for {location}")

    try:
        # API request to fetch weather data
        weather_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={WEATHER_API_KEY}&units=metric"
        weather_data = requests.get(weather_url)
        data = weather_data.json()
        # Extracting relevant weather details
        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]

        # Returning weather details
        return {
            "description": weather_description,
            "temperature": temperature,
            "humidity": humidity
        }
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return {
            "description": "none",
            "temperature": "none",
            "humidity": "none"
        }

In [6]:
def make_api_request(instructions: str) -> str:
    model = LlamaCpp(
        model_path="granite-3.0-8b-instruct-Q4_K_M.gguf", 
        temperature=0,
        min_tokens=5,
        max_tokens=200,
      stop=[TOKENIZER.eos_token]
    )
    response = model.invoke(instructions)
    return response

In [7]:
tools = [get_json_schema(tool) for tool in (get_stock_price, get_current_weather)]
tools

[{'type': 'function',
  'function': {'name': 'get_stock_price',
   'description': 'Retrieves the lowest and highest stock prices for a given ticker and date.',
   'parameters': {'type': 'object',
    'properties': {'ticker': {'type': 'string',
      'description': 'The stock ticker symbol, e.g., "IBM".'},
     'date': {'type': 'string',
      'description': 'The date in "YYYY-MM-DD" format for which you want to get stock prices.'}},
    'required': ['ticker', 'date']},
   'return': {'type': 'object',
    'description': 'A dictionary containing the low and high stock prices on the given date.'}}},
 {'type': 'function',
  'function': {'name': 'get_current_weather',
   'description': 'Fetches the current weather for a given location (default: San Francisco).',
   'parameters': {'type': 'object',
    'properties': {'location': {'type': 'string',
      'description': 'The name of the city for which to retrieve the weather information.'}},
    'required': ['location']},
   'return': {'type':

In [8]:
query = "What were the IBM stock prices on October 7, 2024?"

In [11]:
conversation = [
    {"role": "system","content": "You are a helpful assistant with access to the following function calls. Your task is to produce a list of function calls necessary to generate response to the user utterance. Use the following function calls as required."},
    {"role": "user", "content": query },
]

instruction_1 = TOKENIZER.apply_chat_template(conversation=conversation, tools=tools, tokenize=False, add_generation_prompt=True)
instruction_1

'<|start_of_role|>available_tools<|end_of_role|>\n{\n    "type": "function",\n    "function": {\n        "name": "get_stock_price",\n        "description": "Retrieves the lowest and highest stock prices for a given ticker and date.",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "ticker": {\n                    "type": "string",\n                    "description": "The stock ticker symbol, e.g., \\"IBM\\"."\n                },\n                "date": {\n                    "type": "string",\n                    "description": "The date in \\"YYYY-MM-DD\\" format for which you want to get stock prices."\n                }\n            },\n            "required": [\n                "ticker",\n                "date"\n            ]\n        },\n        "return": {\n            "type": "object",\n            "description": "A dictionary containing the low and high stock prices on the given date."\n        }\n    }\n}\n\n{\n    "type":

In [12]:
data_1 = make_api_request(instruction_1)
data_1

/opt/app-root/lib64/python3.11/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: WARNING! min_tokens is not default parameter.
                min_tokens was transferred to model_kwargs.
                Please confirm that min_tokens is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)
llama_model_loader: loaded meta data with 40 key-value pairs and 362 tensors from granite-3.0-8b-instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = granite
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Granite 3.0 8b Instruct
llama_model_loader: - kv   3:                           general.finetune str              = instruct
llama_model_loader

'[{"name": "get_stock_price", "arguments": {"ticker": "IBM", "date": "2024-10-07"}}]'

In [13]:
def tool_call(llm_response: str):
    tool_request = ast.literal_eval(re.search("({.+})", llm_response).group(0))
    tool_name = tool_request["name"]
    tool_arguments = tool_request["arguments"]
    tool_response = globals()[tool_name](**tool_arguments)
    return tool_response

In [14]:
tool_response = tool_call(data_1)
tool_response

Getting stock price for IBM on 2024-10-07


{'low': '225.0200', 'high': '227.6700'}

In [15]:
conversation2 = conversation + [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Display the tool response in natural language." },
    {"role": "tool_response", "content": str(tool_response) },
]

instruction_2 = TOKENIZER.apply_chat_template(conversation=conversation2, tools=tools, tokenize=False, add_generation_prompt=True)
data_2 = make_api_request(instruction_2)
data_2

/opt/app-root/lib64/python3.11/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: WARNING! min_tokens is not default parameter.
                min_tokens was transferred to model_kwargs.
                Please confirm that min_tokens is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)
llama_model_loader: loaded meta data with 40 key-value pairs and 362 tensors from granite-3.0-8b-instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = granite
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Granite 3.0 8b Instruct
llama_model_loader: - kv   3:                           general.finetune str              = instruct
llama_model_loader

'The IBM stock prices on October 7, 2024 were a low of $225.02 and a high of $227.67.'

In [21]:
query = "What is the current weather in Glasgow, UK?"

In [22]:
conversation = [
    {"role": "system","content": "You are a helpful assistant with access to the following function calls. Your task is to produce a list of function calls necessary to generate response to the user utterance. Use the following function calls as required."},
    {"role": "user", "content": query },
]

instruction_1 = TOKENIZER.apply_chat_template(conversation=conversation, tools=tools, tokenize=False, add_generation_prompt=True)
instruction_1

'<|start_of_role|>available_tools<|end_of_role|>\n{\n    "type": "function",\n    "function": {\n        "name": "get_stock_price",\n        "description": "Retrieves the lowest and highest stock prices for a given ticker and date.",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "ticker": {\n                    "type": "string",\n                    "description": "The stock ticker symbol, e.g., \\"IBM\\"."\n                },\n                "date": {\n                    "type": "string",\n                    "description": "The date in \\"YYYY-MM-DD\\" format for which you want to get stock prices."\n                }\n            },\n            "required": [\n                "ticker",\n                "date"\n            ]\n        },\n        "return": {\n            "type": "object",\n            "description": "A dictionary containing the low and high stock prices on the given date."\n        }\n    }\n}\n\n{\n    "type":

In [23]:
data_1 = make_api_request(instruction_1)
data_1

/opt/app-root/lib64/python3.11/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: WARNING! min_tokens is not default parameter.
                min_tokens was transferred to model_kwargs.
                Please confirm that min_tokens is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)
llama_model_loader: loaded meta data with 40 key-value pairs and 362 tensors from granite-3.0-8b-instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = granite
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Granite 3.0 8b Instruct
llama_model_loader: - kv   3:                           general.finetune str              = instruct
llama_model_loader

'[{"name": "get_current_weather", "arguments": {"location": "Glasgow, UK"}}]'

In [24]:
tool_response = tool_call(data_1)
tool_response

Getting current weather for Glasgow, UK


{'description': 'few clouds', 'temperature': 8.43, 'humidity': 76}

In [25]:
conversation2 = conversation + [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Display the tool response in natural language." },
    {"role": "tool_response", "content": str(tool_response) },
]

instruction_2 = TOKENIZER.apply_chat_template(conversation=conversation2, tools=tools, tokenize=False, add_generation_prompt=True)
data_2 = make_api_request(instruction_2)
data_2

/opt/app-root/lib64/python3.11/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: WARNING! min_tokens is not default parameter.
                min_tokens was transferred to model_kwargs.
                Please confirm that min_tokens is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)
llama_model_loader: loaded meta data with 40 key-value pairs and 362 tensors from granite-3.0-8b-instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = granite
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Granite 3.0 8b Instruct
llama_model_loader: - kv   3:                           general.finetune str              = instruct
llama_model_loader

"The current weather in Glasgow, UK is described as 'few clouds' with a temperature of 8.43 degrees Celsius and humidity at 76%."